In [109]:
import os, sys
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import json

### load original csvs

In [94]:
path_results = '/home/vanveen/ConvDecoder/expmt/results/'
csv_files = [f for f in os.listdir(path_results) if isfile(join(path_results, f))]
csv_paths = [path_results + f for f in csv_files]
df_in = pd.concat((pd.read_csv(f) for f in csv_paths))
df_in

,trial_id,file_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
0,xgzaauhr,1000273,0.7275,29.2965,0.4861,24.7306,0.00000,2000,1600,all,nearest
1,xgzaauhr,1000325,0.6463,27.2535,0.3316,21.2596,0.00000,2000,1600,all,nearest
2,xgzaauhr,1000464,0.7903,29.3034,0.6627,26.4691,0.00000,2000,1600,all,nearest
3,hvldb45v,1000273,0.7287,29.3308,0.4848,24.7901,0.00000,2000,1600,late,bilinear
4,hvldb45v,1000325,0.6477,27.2493,0.3277,21.2318,0.00000,2000,1600,late,bilinear
...,...,...,...,...,...,...,...,...,...,...,...
238,3p9bdztj,1000325,0.6505,27.3481,0.3298,21.2110,0.00100,2000,1000,late,nearest
239,3p9bdztj,1000464,0.7873,29.2242,0.6558,26.3893,0.00100,2000,1000,late,nearest
240,zkshnsfa,1000273,0.7292,29.3032,0.4875,24.7285,0.00001,2000,1600,early,bilinear
241,zkshnsfa,1000325,0.6491,27.2930,0.3373,21.3269,0.00001,2000,1600,early,bilinear


### aggregate trial_ids into same column, i.e. average ssim/psnr across file_ids

In [95]:
display_rows = 10#len(df)
pd.set_option('display.max_rows', display_rows)

In [96]:
def agg_sort_df(df_in):
    ''' given df, aggregate trial_ids across means for different file_ids
        sort df rows by the highest ssim_dc '''
    
    agg_fns = {'ssim_dc': 'mean', 'psnr_dc': 'mean', 'ssim_est':'mean', 'psnr_est':'mean', \
           'alpha_fm': 'first', 'num_iter': 'first', 'iter_start_fm_loss': 'first', \
           'weight_method': 'first', 'downsamp_method':'first'}
    df = df_in.groupby(df_in['trial_id']).aggregate(agg_fns).reset_index()
    df = df.sort_values(by='ssim_dc', ascending=False)
    
    return df

df = agg_sort_df(df_in)
df

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
66,f1g4rdh9,0.728433,28.932900,0.502467,24.481000,0.00010,2000,0,late,nearest
77,i4csenaj,0.727133,28.824767,0.501000,24.403667,0.00010,2000,0,late,bicubic
61,dyn9rycz,0.726833,28.816833,0.496467,24.212033,0.00100,2000,1000,late,bilinear
139,w1v8v9ni,0.726333,28.807200,0.500367,24.465900,0.00001,2000,0,all,bilinear
12,1p0kn9i7,0.725833,28.754067,0.492600,24.087367,0.00010,2000,1600,early,nearest
...,...,...,...,...,...,...,...,...,...,...
128,ui2gux4g,0.649633,26.584367,0.441233,22.162600,0.00100,2000,0,early,bilinear
93,kg9zpja1,0.642900,26.635567,0.413400,20.919067,0.00100,2000,0,all,bicubic
1,0b8qud3y,0.639467,26.834233,0.424267,21.567500,0.00100,2000,0,all,bilinear
30,7jlwlalt,0.634600,25.516700,0.406167,18.853367,0.01000,2000,1000,early,nearest


### check baseline performance, i.e. without using feat_map loss

### confirmed: variance is lowest for `alpha_fm=0`, since that negates all other variables

- TODO: in future, don't need to run this over all variables, since it's the same experiment

In [97]:
alpha_fm_list = list(set(df['alpha_fm'].tolist()))
alpha_fm_list.sort()

for alpha_fm in alpha_fm_list:
    
    df_alpha_fm = df.loc[df['alpha_fm'] == alpha_fm]
    
    if alpha_fm == 0:

        if len(list(set(df_alpha_fm['num_iter'].tolist()))) != 1:
            raise ValueError('current script assumes same num_iter')
        num_iter = 2000
        print('baseline performance for alpha_fm={} on num_iter {}: ssim {}, psnr {}' \
              .format(alpha_fm, num_iter, np.round(df_alpha_fm['ssim_dc'].mean(), 4),
                        np.round(df_alpha_fm['psnr_dc'].mean(), 4)))
        
    print(alpha_fm, np.round(df_alpha_fm['ssim_dc'].std(), 4))

baseline performance for alpha_fm=0.0 on num_iter 2000: ssim 0.722, psnr 28.6451
0.0 0.0009
1e-05 0.0017
0.0001 0.0028
0.001 0.0305
0.01 0.0257
0.1 0.0277


### get best value for each variable

finding: given std() of each individual variable, too difficult to say whether one value performs better than another with confidence

In [98]:
variable_list = ['alpha_fm', 'iter_start_fm_loss', 'weight_method', 'downsamp_method']

for var in variable_list:
    
    print(var)
    var_values = list(set(df[var].tolist()))
    var_values.sort()
    
    for value in var_values:
        
        df_val = df.loc[df[var] == value]
        print('{}: ssim ~ N({}, {}), psnr ~ N({}, {})'.format(value, \
                     np.round(df_val['ssim_dc'].mean(), 4), \
                     np.round(df_val['ssim_dc'].std(), 4), \
                     np.round(df_val['psnr_dc'].mean(), 4), \
                     np.round(df_val['psnr_dc'].std(), 4)))
        
    print('')

alpha_fm
0.0: ssim ~ N(0.722, 0.0009), psnr ~ N(28.6451, 0.0342)
1e-05: ssim ~ N(0.7224, 0.0017), psnr ~ N(28.635, 0.0679)
0.0001: ssim ~ N(0.7226, 0.0028), psnr ~ N(28.6512, 0.1112)
0.001: ssim ~ N(0.7036, 0.0305), psnr ~ N(28.1225, 0.8239)
0.01: ssim ~ N(0.7051, 0.0257), psnr ~ N(28.164, 0.7131)
0.1: ssim ~ N(0.7026, 0.0277), psnr ~ N(28.0624, 0.7551)

iter_start_fm_loss
0: ssim ~ N(0.705, 0.0267), psnr ~ N(28.2254, 0.6605)
1000: ssim ~ N(0.7138, 0.0237), psnr ~ N(28.3561, 0.7333)
1600: ssim ~ N(0.7202, 0.0061), psnr ~ N(28.5586, 0.2344)

weight_method
all: ssim ~ N(0.7122, 0.021), psnr ~ N(28.3588, 0.5693)
early: ssim ~ N(0.7065, 0.029), psnr ~ N(28.2107, 0.7921)
late: ssim ~ N(0.7204, 0.0067), psnr ~ N(28.5706, 0.2613)

downsamp_method
bicubic: ssim ~ N(0.7125, 0.0207), psnr ~ N(28.3436, 0.6229)
bilinear: ssim ~ N(0.7122, 0.0257), psnr ~ N(28.3794, 0.6331)
nearest: ssim ~ N(0.7144, 0.0182), psnr ~ N(28.4172, 0.5429)



### questions to answer

- given top trial_ids, if i run the same values over more than three samples, will we get the same results?
    - i.e. evaluate statistical significance of three samples
    - running via `search_hyperparam_larger_set.py` / `train_larger_set.py`
- if i run this over 10000 iter instead of 2000 iter, will the results be the same?
    - i.e. evaluate statistical significance of 2000 iter
    - running via `search_hyperparam_orig_three.py` / `train_orig_three.py`
    

Note: original base script for expmts is `search_hyperparam.py` / `train.py`

# TODO: look at results

In [122]:
df_best = df[:20]
df_best

,trial_id,ssim_dc,psnr_dc,ssim_est,psnr_est,alpha_fm,num_iter,iter_start_fm_loss,weight_method,downsamp_method
66,f1g4rdh9,0.728433,28.932900,0.502467,24.481000,0.00010,2000,0,late,nearest
77,i4csenaj,0.727133,28.824767,0.501000,24.403667,0.00010,2000,0,late,bicubic
61,dyn9rycz,0.726833,28.816833,0.496467,24.212033,0.00100,2000,1000,late,bilinear
139,w1v8v9ni,0.726333,28.807200,0.500367,24.465900,0.00001,2000,0,all,bilinear
12,1p0kn9i7,0.725833,28.754067,0.492600,24.087367,0.00010,2000,1600,early,nearest
...,...,...,...,...,...,...,...,...,...,...
107,r71wfg6k,0.724067,28.678900,0.502433,24.376533,0.00001,2000,0,all,nearest
91,k6aoqok1,0.724067,28.682733,0.494200,24.168800,0.00001,2000,1000,early,bicubic
96,nk28r9f1,0.724000,28.702033,0.492700,24.112233,0.00001,2000,1000,all,nearest
130,uvz74b9u,0.723967,28.756000,0.489667,24.032433,0.00010,2000,0,early,bicubic


In [123]:
dict_best_trials = df_best.to_dict('index')

json_path = path_results + 'trials_best_20201121.json'

with open(json_path, 'w') as f:
    json.dump(dict_best_trials, f)
    
with open(json_path, 'r') as f:
    dict_best_trials = json.load(f)
    
dict_best_trials

{66: {'trial_id': 'f1g4rdh9',
  'ssim_dc': 0.7284333333333333,
  'psnr_dc': 28.9329,
  'ssim_est': 0.5024666666666667,
  'psnr_est': 24.480999999999998,
  'alpha_fm': 0.0001,
  'num_iter': 2000,
  'iter_start_fm_loss': 0,
  'weight_method': 'late',
  'downsamp_method': 'nearest'},
 77: {'trial_id': 'i4csenaj',
  'ssim_dc': 0.7271333333333333,
  'psnr_dc': 28.824766666666665,
  'ssim_est': 0.501,
  'psnr_est': 24.403666666666666,
  'alpha_fm': 0.0001,
  'num_iter': 2000,
  'iter_start_fm_loss': 0,
  'weight_method': 'late',
  'downsamp_method': 'bicubic'},
 61: {'trial_id': 'dyn9rycz',
  'ssim_dc': 0.7268333333333333,
  'psnr_dc': 28.816833333333335,
  'ssim_est': 0.49646666666666667,
  'psnr_est': 24.212033333333334,
  'alpha_fm': 0.001,
  'num_iter': 2000,
  'iter_start_fm_loss': 1000,
  'weight_method': 'late',
  'downsamp_method': 'bilinear'},
 139: {'trial_id': 'w1v8v9ni',
  'ssim_dc': 0.7263333333333334,
  'psnr_dc': 28.807199999999998,
  'ssim_est': 0.5003666666666667,
  'psnr_es